In [1]:
import os
import re
import sys

import numpy as np
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics as skmetrics
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.font_manager import fontManager
from IPython.display import display
from ipywidgets import interact, FloatSlider
from umap import UMAP
import ipynbname
from endaaman.ml import fix_global_seed
from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)
fix_global_seed()

project_root = os.path.abspath(os.path.join(os.path.dirname(ipynbname.path()), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
os.chdir(project_root)
print('now working', project_root)


sns.set_theme(style='ticks', palette='tab10')
plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 16,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'legend.fontsize': 16,
    'figure.dpi': 300,
})

os.makedirs('out/figs', exist_ok=True)

now working /home/ken/src/github.com/endaaman/bt


NameError: name 'init_notebook_mode' is not defined

## Table2: Cross Validation

In [2]:
mm = [
    'Macro Recall', 
    'Patch Acc.', 
    'Macro Acc.', 
    'Macro F1', 
    'G Recall', 
    'A Recall', 
    'O Recall', 
    'M Recall', 
    'L Recall', 

    'G Prec.', 
    'A Prec.', 
    'O Prec.', 
    'M Prec.', 
    'L Prec.',     
]

def get_overall_acc(target, cond, coarse):
    if target == 'cv':
        dfs = []
        for fold in range(5):
            df = pd.read_excel(f'out/compare/LMGAOB/fold5_{fold}/{cond}/test/report.xlsx', sheet_name='cases', index_col=0)
            dfs.append(df)
        df = pd.concat(dfs)
        df_B = df[df['diag_org'] == 'B']        
        
        df = df[df['diag_org'] != 'B']        
        if coarse:
            f = df[df['correct3'] > 0]
        else:
            f = df[df['correct'] > 0]
        return f'{len(f)+5}/{len(df)+5}'

    dfs = []
    for fold in range(5):
        df = pd.read_excel(f'out/compare/LMGAOB/fold5_{fold}/{cond}/ebrains.xlsx', sheet_name='cases', index_col=0)
        dfs.append(df)
    df = pd.concat(dfs)    
    unique_codes = list('LMGAOB')
    map3= {
        'L': 'L',
        'M': 'M',
        'G': 'G',
        'A': 'G',
        'O': 'G',
    }
    data = []
    for name, rows in df.groupby('name'):
        gt = rows.iloc[0]['label']
        preds = []
        for c in unique_codes:
            preds.append(rows[c].mean())
        # label_index = np.argmax(preds)
        pred_ordered = np.array(unique_codes)[np.argsort(preds)[::-1]]
        pred = pred_ordered[1] if pred_ordered[0] == 'B' else pred_ordered[0]    
        data.append({
            'name': name,
            'gt': gt,
            'pred': pred,
            'correct': gt == pred,
            'correct3': map3[gt] == map3[pred],
        })
    df = pd.DataFrame(data)
    if coarse:
        f = df[df['correct3'] > 0]
    else:
        f = df[df['correct'] > 0]
    return f'{len(f)}/{len(df)}'



skip_when_coarse = ['A Recall', 'O Recall', 'A Prec.', 'O Prec.']

def dump_to_table(target, coarse=False, result_limit=100):
    r = None
    error_type = 'ci'
    grains = 'coarse' if coarse else 'fine'
    P = f'out/figs/tables/score_{grains}_{target}.xlsx'
    with pd.ExcelWriter(P) as w:
        for limit in [500, 100, 25, 10]:
            df = pd.read_excel(f'out/figs/tables/results_{grains}_{target}.xlsx', sheet_name=f'{limit}', index_col=0)
            data = []
            labels = df['label'].unique()
            conds = df['cond'].unique()
            for l, cond in zip(labels, conds):
                vvv = df[df['label'] == l]
                ss = {}
                ss['Overall Acc.'] = get_overall_acc(target, cond, coarse)
                for m in mm:
                    if coarse and m in skip_when_coarse:
                        continue
                    vv = vvv[m].values
                    mean = np.mean(vv)
                    if len(vv) > 0 and vv.std() > 0:
                        if error_type == 'ci':
                            ci = stats.t.interval(confidence=0.95, df=len(vv)-1, loc=mean, scale=stats.sem(vv))
                            error = (ci[1] - ci[0])/2
                        else:
                            error = stats.sem(vv)
                    else:
                        error = 0.0
                    s = '{:.2f} ± {:.2f}'.format(mean, error)
                    ss[m] = s
                data.append(ss)
            df2 = pd.DataFrame(data, index=labels)
            df2.to_excel(w, sheet_name=f'{limit}')
            w.sheets[f'{limit}'].set_column('A:A', 30)
            w.sheets[f'{limit}'].set_column('B:Z', 12)
            if limit == result_limit:
                print(f'return {result_limit}')
                r = df2
    print(f'wrote {P}')
    return r

## Table2: Local cross-validation

In [3]:
dump_to_table('cv', False, LIMIT)

return 500
wrote out/figs/tables/score_fine_cv.xlsx


Overall Acc. Macro Recall   Patch Acc.  \
Prov-GigaPath(LP)                     234/254  0.88 ± 0.10  0.93 ± 0.02   
UNI(FT)                               231/254  0.87 ± 0.07  0.94 ± 0.01   
UNI(LP)                               234/254  0.88 ± 0.09  0.93 ± 0.03   
CTransPath(FT)                        233/254  0.88 ± 0.09  0.92 ± 0.02   
CTransPath(LP)                        230/254  0.87 ± 0.06  0.90 ± 0.03   
ViT-L$\mathrm{_{IN}}$(FT)             225/254  0.83 ± 0.08  0.91 ± 0.02   
ViT-L$\mathrm{_{IN}}$(LP)             217/254  0.83 ± 0.05  0.86 ± 0.03   
ViT-L(RI)                             215/254  0.83 ± 0.04  0.86 ± 0.02   
ResNet-RS 50$\mathrm{_{IN}}$(FT)      229/254  0.85 ± 0.10  0.91 ± 0.02   
ResNet-RS 50$\mathrm{_{IN}}$(LP)       57/254  0.36 ± 0.07  0.38 ± 0.09   

                                   Macro Acc.     Macro F1     G Recall  \
Prov-GigaPath(LP)                 0.95 ± 0.04  0.88 ± 0.10  0.95 ± 0.05   
UNI(FT)                           0.94 ± 0.03  0.86 ± 0.09  0.98 ± 0.04   
UNI(LP)                           0.95 ± 0.04  0.88 ± 0.10  0.97 ± 0.05   
CTransPath(FT)                    0.95 ± 0.04  0.88 ± 0.09  0.97 ± 0.05   
CTransPath(LP)                    0.94 ± 0.02  0.87 ± 0.05  0.94 ± 0.12   
ViT-L$\mathrm{_{IN}}$(FT)         0.93 ± 0.03  0.83 ± 0.09  0.95 ± 0.09   
ViT-L$\mathrm{_{IN}}$(LP)         0.91 ± 0.02  0.83 ± 0.05  0.85 ± 0.19   
ViT-L(RI)                         0.90 ± 0.03  0.83 ± 0.05  0.92 ± 0.07   
ResNet-RS 50$\mathrm{_{IN}}$(FT)  0.94 ± 0.04  0.84 ± 0.11  0.98 ± 0.04   
ResNet-RS 50$\mathrm{_{IN}}$(LP)  0.37 ± 0.12  0.23 ± 0.06  0.00 ± 0.00   

                                     A Recall     O Recall     M Recall  \
Prov-GigaPath(LP)                 0.53 ± 0.46  0.82 ± 0.25  0.99 ± 0.03   
UNI(FT)                           0.39 ± 0.35  0.87 ± 0.15  0.98 ± 0.04   
UNI(LP)                           0.49 ± 0.42  0.87 ± 0.15  0.99 ± 0.03   
CTransPath(FT)                    0.46 ± 0.41  0.87 ± 0.15  0.98 ± 0.04   
CTransPath(LP)                    0.47 ± 0.38  0.87 ± 0.15  0.99 ± 0.03   
ViT-L$\mathrm{_{IN}}$(FT)         0.34 ± 0.30  0.75 ± 0.27  0.97 ± 0.07   
ViT-L$\mathrm{_{IN}}$(LP)         0.39 ± 0.25  0.87 ± 0.15  0.95 ± 0.06   
ViT-L(RI)                         0.58 ± 0.31  0.71 ± 0.37  0.90 ± 0.08   
ResNet-RS 50$\mathrm{_{IN}}$(FT)  0.35 ± 0.51  0.83 ± 0.12  0.97 ± 0.04   
ResNet-RS 50$\mathrm{_{IN}}$(LP)  0.00 ± 0.00  0.92 ± 0.14  0.00 ± 0.00   

                                     L Recall      G Prec.      A Prec.  \
Prov-GigaPath(LP)                 1.00 ± 0.00  0.87 ± 0.11  0.77 ± 0.31   
UNI(FT)                           0.98 ± 0.06  0.87 ± 0.12  0.73 ± 0.54   
UNI(LP)                           0.98 ± 0.06  0.86 ± 0.12  0.83 ± 0.29   
CTransPath(FT)                    1.00 ± 0.00  0.88 ± 0.11  0.83 ± 0.29   
CTransPath(LP)                    0.96 ± 0.07  0.90 ± 0.09  0.73 ± 0.24   
ViT-L$\mathrm{_{IN}}$(FT)         0.98 ± 0.06  0.81 ± 0.14  0.63 ± 0.52   
ViT-L$\mathrm{_{IN}}$(LP)         0.92 ± 0.10  0.86 ± 0.10  0.67 ± 0.26   
ViT-L(RI)                         0.84 ± 0.11  0.76 ± 0.11  0.73 ± 0.21   
ResNet-RS 50$\mathrm{_{IN}}$(FT)  0.98 ± 0.06  0.82 ± 0.15  0.53 ± 0.63   
ResNet-RS 50$\mathrm{_{IN}}$(LP)  0.57 ± 0.36  0.00 ± 0.00  0.00 ± 0.00   

                                      O Prec.      M Prec.      L Prec.  
Prov-GigaPath(LP)                 0.86 ± 0.18  0.99 ± 0.03  0.98 ± 0.05  
UNI(FT)                           0.75 ± 0.24  1.00 ± 0.00  0.97 ± 0.06  
UNI(LP)                           0.83 ± 0.21  0.99 ± 0.03  0.98 ± 0.05  
CTransPath(FT)                    0.79 ± 0.25  1.00 ± 0.00  0.95 ± 0.09  
CTransPath(LP)                    0.73 ± 0.09  0.98 ± 0.06  1.00 ± 0.00  
ViT-L$\mathrm{_{IN}}$(FT)         0.79 ± 0.19  0.99 ± 0.03  0.96 ± 0.06  
ViT-L$\mathrm{_{IN}}$(LP)         0.80 ± 0.27  0.93 ± 0.09  0.86 ± 0.09  
ViT-L(RI)                         0.89 ± 0.19  0.93 ± 0.06  0.98 ± 0.06  
ResNet-RS 50$\mathrm{_{IN}}$(FT)  0.81 ± 0.26  1.00 ± 0.00  0.9

In [4]:
dump_to_table('ebrains', False, LIMIT)

return 500
wrote out/figs/tables/score_fine_ebrains.xlsx


Overall Acc. Macro Recall   Patch Acc.  \
Prov-GigaPath(LP)                     474/698  0.73 ± 0.01  0.64 ± 0.01   
UNI(FT)                               470/698  0.71 ± 0.02  0.64 ± 0.02   
UNI(LP)                               479/698  0.75 ± 0.01  0.65 ± 0.02   
CTransPath(FT)                        437/698  0.68 ± 0.01  0.59 ± 0.03   
CTransPath(LP)                        446/698  0.69 ± 0.01  0.59 ± 0.01   
ViT-L$\mathrm{_{IN}}$(FT)             422/698  0.64 ± 0.02  0.56 ± 0.04   
ViT-L$\mathrm{_{IN}}$(LP)             407/698  0.64 ± 0.02  0.53 ± 0.04   
ViT-L(RI)                             336/698  0.54 ± 0.04  0.44 ± 0.05   
ResNet-RS 50$\mathrm{_{IN}}$(FT)      416/698  0.62 ± 0.03  0.54 ± 0.03   
ResNet-RS 50$\mathrm{_{IN}}$(LP)      189/698  0.29 ± 0.04  0.13 ± 0.04   

                                   Macro Acc.     Macro F1     G Recall  \
Prov-GigaPath(LP)                 0.67 ± 0.02  0.67 ± 0.02  0.74 ± 0.04   
UNI(FT)                           0.65 ± 0.03  0.64 ± 0.04  0.76 ± 0.03   
UNI(LP)                           0.68 ± 0.01  0.70 ± 0.02  0.73 ± 0.07   
CTransPath(FT)                    0.61 ± 0.03  0.62 ± 0.03  0.71 ± 0.06   
CTransPath(LP)                    0.62 ± 0.01  0.64 ± 0.01  0.68 ± 0.03   
ViT-L$\mathrm{_{IN}}$(FT)         0.58 ± 0.03  0.56 ± 0.04  0.74 ± 0.09   
ViT-L$\mathrm{_{IN}}$(LP)         0.58 ± 0.04  0.55 ± 0.04  0.69 ± 0.11   
ViT-L(RI)                         0.47 ± 0.06  0.45 ± 0.06  0.58 ± 0.12   
ResNet-RS 50$\mathrm{_{IN}}$(FT)  0.56 ± 0.04  0.54 ± 0.05  0.70 ± 0.09   
ResNet-RS 50$\mathrm{_{IN}}$(LP)  0.27 ± 0.02  0.17 ± 0.03  0.00 ± 0.00   

                                     A Recall     O Recall     M Recall  \
Prov-GigaPath(LP)                 0.61 ± 0.06  0.45 ± 0.05  0.90 ± 0.02   
UNI(FT)                           0.34 ± 0.11  0.59 ± 0.13  0.89 ± 0.01   
UNI(LP)                           0.59 ± 0.06  0.52 ± 0.05  0.95 ± 0.01   
CTransPath(FT)                    0.44 ± 0.05  0.42 ± 0.06  0.88 ± 0.04   
CTransPath(LP)                    0.58 ± 0.08  0.41 ± 0.08  0.89 ± 0.01   
ViT-L$\mathrm{_{IN}}$(FT)         0.27 ± 0.11  0.42 ± 0.10  0.86 ± 0.05   
ViT-L$\mathrm{_{IN}}$(LP)         0.41 ± 0.22  0.36 ± 0.26  0.86 ± 0.07   
ViT-L(RI)                         0.27 ± 0.09  0.30 ± 0.09  0.75 ± 0.05   
ResNet-RS 50$\mathrm{_{IN}}$(FT)  0.26 ± 0.09  0.40 ± 0.06  0.84 ± 0.03   
ResNet-RS 50$\mathrm{_{IN}}$(LP)  0.09 ± 0.13  0.83 ± 0.12  0.00 ± 0.00   

                                     L Recall      G Prec.      A Prec.  \
Prov-GigaPath(LP)                 0.95 ± 0.00  0.78 ± 0.03  0.48 ± 0.02   
UNI(FT)                           0.96 ± 0.01  0.72 ± 0.03  0.57 ± 0.06   
UNI(LP)                           0.96 ± 0.02  0.77 ± 0.03  0.48 ± 0.02   
CTransPath(FT)                    0.94 ± 0.02  0.65 ± 0.03  0.47 ± 0.03   
CTransPath(LP)                    0.91 ± 0.04  0.72 ± 0.03  0.44 ± 0.04   
ViT-L$\mathrm{_{IN}}$(FT)         0.89 ± 0.04  0.63 ± 0.05  0.51 ± 0.05   
ViT-L$\mathrm{_{IN}}$(LP)         0.87 ± 0.04  0.72 ± 0.07  0.51 ± 0.09   
ViT-L(RI)                         0.79 ± 0.09  0.58 ± 0.04  0.44 ± 0.07   
ResNet-RS 50$\mathrm{_{IN}}$(FT)  0.90 ± 0.06  0.60 ± 0.01  0.45 ± 0.07   
ResNet-RS 50$\mathrm{_{IN}}$(LP)  0.52 ± 0.22  0.93 ± 0.19  0.27 ± 0.29   

                                      O Prec.      M Prec.      L Prec.  
Prov-GigaPath(LP)                 0.85 ± 0.06  0.61 ± 0.08  0.62 ± 0.04  
UNI(FT)                           0.69 ± 0.09  0.61 ± 0.07  0.55 ± 0.14  
UNI(LP)                           0.83 ± 0.06  0.68 ± 0.07  0.66 ± 0.05  
CTransPath(FT)                    0.75 ± 0.05  0.57 ± 0.07  0.58 ± 0.04  
CTransPath(LP)                    0.82 ± 0.04  0.56 ± 0.06  0.66 ± 0.05  
ViT-L$\mathrm{_{IN}}$(FT)         0.68 ± 0.13  0.52 ± 0.15  0.48 ± 0.05  
ViT-L$\mathrm{_{IN}}$(LP)         0.80 ± 0.16  0.50 ± 0.12  0.37 ± 0.04  
ViT-L(RI)                         0.56 ± 0.05  0.42 ± 0.10  0.30 ± 0.08  
ResNet-RS 50$\mathrm{_{IN}}$(FT)  0.71 ± 0.08  0.55 ± 0.15  0.4

## Table2: EBRAINS

In [5]:
dump_to_table('cv', True, LIMIT)

return 500
wrote out/figs/tables/score_coarse_cv.xlsx


Overall Acc. Macro Recall   Patch Acc.  \
Prov-GigaPath(LP)                     251/254  0.99 ± 0.01  0.97 ± 0.01   
UNI(FT)                               250/254  0.99 ± 0.02  0.97 ± 0.01   
UNI(LP)                               250/254  0.99 ± 0.02  0.97 ± 0.01   
CTransPath(FT)                        250/254  0.99 ± 0.02  0.96 ± 0.01   
CTransPath(LP)                        248/254  0.98 ± 0.02  0.95 ± 0.01   
ViT-L$\mathrm{_{IN}}$(FT)             248/254  0.98 ± 0.02  0.95 ± 0.02   
ViT-L$\mathrm{_{IN}}$(LP)             235/254  0.94 ± 0.02  0.91 ± 0.02   
ViT-L(RI)                             233/254  0.93 ± 0.04  0.91 ± 0.02   
ResNet-RS 50$\mathrm{_{IN}}$(FT)      250/254  0.99 ± 0.01  0.96 ± 0.01   
ResNet-RS 50$\mathrm{_{IN}}$(LP)      126/254  0.50 ± 0.09  0.52 ± 0.06   

                                   Macro Acc.     Macro F1     G Recall  \
Prov-GigaPath(LP)                 0.99 ± 0.01  0.99 ± 0.02  0.98 ± 0.03   
UNI(FT)                           0.99 ± 0.01  0.99 ± 0.02  0.99 ± 0.02   
UNI(LP)                           0.99 ± 0.01  0.99 ± 0.02  0.98 ± 0.03   
CTransPath(FT)                    0.99 ± 0.02  0.99 ± 0.02  0.98 ± 0.03   
CTransPath(LP)                    0.98 ± 0.01  0.98 ± 0.01  0.97 ± 0.05   
ViT-L$\mathrm{_{IN}}$(FT)         0.98 ± 0.01  0.98 ± 0.01  0.97 ± 0.05   
ViT-L$\mathrm{_{IN}}$(LP)         0.95 ± 0.02  0.94 ± 0.03  0.90 ± 0.10   
ViT-L(RI)                         0.95 ± 0.03  0.93 ± 0.04  0.96 ± 0.00   
ResNet-RS 50$\mathrm{_{IN}}$(FT)  0.99 ± 0.01  0.99 ± 0.01  0.99 ± 0.02   
ResNet-RS 50$\mathrm{_{IN}}$(LP)  0.54 ± 0.08  0.44 ± 0.08  0.78 ± 0.19   

                                     M Recall     L Recall      G Prec.  \
Prov-GigaPath(LP)                 0.99 ± 0.03  1.00 ± 0.00  0.99 ± 0.02   
UNI(FT)                           0.98 ± 0.04  0.98 ± 0.06  0.98 ± 0.03   
UNI(LP)                           0.99 ± 0.03  0.98 ± 0.06  0.98 ± 0.03   
CTransPath(FT)                    0.98 ± 0.04  1.00 ± 0.00  0.99 ± 0.02   
CTransPath(LP)                    0.99 ± 0.03  0.96 ± 0.07  0.98 ± 0.03   
ViT-L$\mathrm{_{IN}}$(FT)         0.97 ± 0.07  0.98 ± 0.06  0.98 ± 0.04   
ViT-L$\mathrm{_{IN}}$(LP)         0.95 ± 0.06  0.92 ± 0.10  0.97 ± 0.04   
ViT-L(RI)                         0.90 ± 0.08  0.84 ± 0.11  0.90 ± 0.06   
ResNet-RS 50$\mathrm{_{IN}}$(FT)  0.97 ± 0.04  0.98 ± 0.06  0.98 ± 0.03   
ResNet-RS 50$\mathrm{_{IN}}$(LP)  0.00 ± 0.00  0.57 ± 0.36  0.43 ± 0.09   

                                      M Prec.      L Prec.  
Prov-GigaPath(LP)                 0.99 ± 0.03  0.98 ± 0.05  
UNI(FT)                           1.00 ± 0.00  0.97 ± 0.06  
UNI(LP)                           0.99 ± 0.03  0.98 ± 0.05  
CTransPath(FT)                    1.00 ± 0.00  0.95 ± 0.09  
CTransPath(LP)                    0.98 ± 0.06  1.00 ± 0.00  
ViT-L$\mathrm{_{IN}}$(FT)         0.99 ± 0.03  0.96 ± 0.06  
ViT-L$\mathrm{_{IN}}$(LP)         0.93 ± 0.09  0.86 ± 0.09  
ViT-L(RI)                         0.93 ± 0.06  0.98 ± 0.06  
ResNet-RS 50$\mathrm{_{IN}}$(FT)  1.00 ± 0.00  0.98 ± 0.05  
ResNet-RS 50$\mathrm{_{IN}}$(LP)  0.00 ± 0.00  0.57 ± 0.12

In [6]:
dump_to_table('ebrains', True, LIMIT)

return 500
wrote out/figs/tables/score_coarse_ebrains.xlsx


Overall Acc. Macro Recall   Patch Acc.  \
Prov-GigaPath(LP)                     635/698  0.92 ± 0.00  0.88 ± 0.01   
UNI(FT)                               625/698  0.91 ± 0.02  0.87 ± 0.04   
UNI(LP)                               648/698  0.94 ± 0.01  0.90 ± 0.02   
CTransPath(FT)                        626/698  0.90 ± 0.01  0.86 ± 0.03   
CTransPath(LP)                        632/698  0.90 ± 0.01  0.85 ± 0.02   
ViT-L$\mathrm{_{IN}}$(FT)             602/698  0.87 ± 0.02  0.81 ± 0.05   
ViT-L$\mathrm{_{IN}}$(LP)             561/698  0.84 ± 0.02  0.74 ± 0.05   
ViT-L(RI)                             537/698  0.76 ± 0.03  0.69 ± 0.06   
ResNet-RS 50$\mathrm{_{IN}}$(FT)      605/698  0.86 ± 0.01  0.79 ± 0.03   
ResNet-RS 50$\mathrm{_{IN}}$(LP)      549/698  0.45 ± 0.04  0.46 ± 0.16   

                                   Macro Acc.     Macro F1     G Recall  \
Prov-GigaPath(LP)                 0.91 ± 0.01  0.81 ± 0.02  0.90 ± 0.02   
UNI(FT)                           0.88 ± 0.04  0.78 ± 0.06  0.88 ± 0.05   
UNI(LP)                           0.92 ± 0.02  0.84 ± 0.03  0.92 ± 0.02   
CTransPath(FT)                    0.89 ± 0.02  0.78 ± 0.02  0.88 ± 0.03   
CTransPath(LP)                    0.90 ± 0.02  0.80 ± 0.03  0.91 ± 0.02   
ViT-L$\mathrm{_{IN}}$(FT)         0.85 ± 0.04  0.73 ± 0.05  0.85 ± 0.06   
ViT-L$\mathrm{_{IN}}$(LP)         0.80 ± 0.03  0.67 ± 0.03  0.79 ± 0.04   
ViT-L(RI)                         0.76 ± 0.08  0.61 ± 0.07  0.75 ± 0.09   
ResNet-RS 50$\mathrm{_{IN}}$(FT)  0.84 ± 0.03  0.71 ± 0.05  0.83 ± 0.04   
ResNet-RS 50$\mathrm{_{IN}}$(LP)  0.76 ± 0.07  0.39 ± 0.02  0.84 ± 0.10   

                                     M Recall     L Recall      G Prec.  \
Prov-GigaPath(LP)                 0.90 ± 0.02  0.95 ± 0.00  0.99 ± 0.00   
UNI(FT)                           0.89 ± 0.01  0.96 ± 0.01  0.99 ± 0.00   
UNI(LP)                           0.95 ± 0.01  0.96 ± 0.02  1.00 ± 0.00   
CTransPath(FT)                    0.88 ± 0.04  0.94 ± 0.02  0.99 ± 0.00   
CTransPath(LP)                    0.89 ± 0.01  0.91 ± 0.04  0.99 ± 0.00   
ViT-L$\mathrm{_{IN}}$(FT)         0.86 ± 0.05  0.89 ± 0.04  0.99 ± 0.01   
ViT-L$\mathrm{_{IN}}$(LP)         0.86 ± 0.07  0.87 ± 0.04  0.99 ± 0.01   
ViT-L(RI)                         0.75 ± 0.05  0.79 ± 0.09  0.98 ± 0.01   
ResNet-RS 50$\mathrm{_{IN}}$(FT)  0.84 ± 0.03  0.90 ± 0.06  0.99 ± 0.00   
ResNet-RS 50$\mathrm{_{IN}}$(LP)  0.00 ± 0.00  0.52 ± 0.22  0.89 ± 0.02   

                                      M Prec.      L Prec.  
Prov-GigaPath(LP)                 0.61 ± 0.08  0.62 ± 0.04  
UNI(FT)                           0.61 ± 0.07  0.55 ± 0.14  
UNI(LP)                           0.68 ± 0.07  0.66 ± 0.05  
CTransPath(FT)                    0.57 ± 0.07  0.58 ± 0.04  
CTransPath(LP)                    0.56 ± 0.06  0.66 ± 0.05  
ViT-L$\mathrm{_{IN}}$(FT)         0.52 ± 0.15  0.48 ± 0.05  
ViT-L$\mathrm{_{IN}}$(LP)         0.50 ± 0.12  0.37 ± 0.04  
ViT-L(RI)                         0.42 ± 0.10  0.30 ± 0.08  
ResNet-RS 50$\mathrm{_{IN}}$(FT)  0.55 ± 0.15  0.43 ± 0.07  
ResNet-RS 50$\mathrm{_{IN}}$(LP)  1.00 ± 0.00  0.23 ± 0.05

## fig3: Box - Cross Validation

In [7]:
import random
import colorcet as cc

labels = [
    'Prov-GigaPath(LP)',
    'UNI(FT)',
    'UNI(LP)',
    'CTransPath(FT)',
    'CTransPath(LP)',
    r'ViT-L$\mathrm{_{IN}}$(FT)',
    r'ViT-L$\mathrm{_{IN}}$(LP)',
    r'ViT-L(RI)',
    r'ResNet-RS 50$\mathrm{_{IN}}$(FT)',
    r'ResNet-RS 50$\mathrm{_{IN}}$(LP)',
    r'ResNet-RS 50(RI)',
]
print(len(labels))
# colors = sns.color_palette('pastel20', len(label))
colors = sns.color_palette(cc.glasbey_hv, n_colors=len(labels))
color_map = dict(zip(labels, colors))
display(colors)

11


[(0.18824, 0.63529, 0.85490),
 (0.98823, 0.30980, 0.18824),
 (0.89804, 0.68235, 0.21961),
 (0.42745, 0.56471, 0.30980),
 (0.54510, 0.54510, 0.54510),
 (0.09020, 0.74510, 0.81176),
 (0.58039, 0.40392, 0.74118),
 (0.83922, 0.15294, 0.15686),
 (0.12157, 0.46667, 0.70588),
 (0.89020, 0.46667, 0.76078),
 (0.54902, 0.33726, 0.29412)]

In [8]:
from statsmodels.stats.multitest import multipletests

verbose = False

def paired_box(target:str, 
               coarse:bool, 
               limit:int, 
               key:str, 
               show:bool=False, 
               excludes=['frozen_baseline-cnn'],
               sort=False,
               mean=False,
               log_scale=False,
              ):
    grains = 'coarse' if coarse else 'fine'
    key_map = {
        'pacc': 'Patch Acc.',
        'recall': 'Macro Recall',
        'acc': 'Macro Acc.',
        'f1': 'Macro F1',
    }
    key_label = key_map[key]
    
    df = pd.read_excel(f'out/figs/tables/results_{grains}_{target}.xlsx', sheet_name=f'{limit}', index_col=0)    
    for e in excludes:
        df = df[~df['cond'].str.startswith(e)]
    labels = list(df['label'].unique())

    n_folds, n_models = 5, len(df)//5

    df_mean = df.groupby('cond')[key_label].mean().sort_values(ascending=True)
    df[f'average_{key}'] = df['cond'].map(df_mean)
    if sort:
        df = df.sort_values([f'average_{key}', 'fold'], ascending=[True, True])
    else:
        df = df[::-1]    
    labels_sorted = df['label'].unique()

    # Create figure with GridSpec
    fig = plt.figure(figsize=(8, 10))
    gs = fig.add_gridspec(2, 2, width_ratios=[20, 1], height_ratios=[3, 1], hspace=0.5, wspace=0.2)

    # Create axes
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[1, 0])
    cax = fig.add_subplot(gs[1, 1])

    # Box plot
    
    sns.boxplot(x='label', hue='label', y=key_label, data=df, legend=True, palette=[color_map[label] for label in labels_sorted], ax=ax1)

    # Connect folds with lines
    for fold in range(n_folds):
        fold_data = df[df['fold'] == fold]
        ax1.plot(range(n_models), fold_data[key_label], 'k-', alpha=0.3, linewidth=0.5)
    
    # Scatter individual points
    for i, label in enumerate(labels_sorted):
        ax1.scatter([i] * n_folds, df[df['label'] == label][key_label], color='black', s=10, zorder=3)
    
    # Plot mean
    if mean:
        ax1.plot(range(n_models), df_mean, 'firebrick', linewidth=1, zorder=4, alpha=0.4)

    mean_scatter = ax1.scatter(
        labels, 
        [df[df['label'] == l][key_label].mean() for l in labels],
        color='firebrick', s=50, zorder=5, label='Mean', alpha=0.7)

    # Set ax1 properties
    target_dataset = {
        'cv': 'Local',
        'ebrains': 'EBRAINS',
    }[target]
    ax1.set_title(f'{target_dataset} - {key_label}')
    ax1.set_xticks(range(len(labels_sorted)))
    ax1.set_xticklabels(labels_sorted, rotation=45, ha='right')
    ax1.set_xlabel('')
    ax1.set_ylabel(key_label)


    # Calculate p-values
    p_values = []
    for i in range(n_models):
        for j in range(i+1, n_models):
            a = df[df['label']==labels_sorted[i]][key_label]
            b = df[df['label']==labels_sorted[j]][key_label]
            t_stat, p_value = stats.ttest_rel(a, b)
            p_values.append(p_value)

    # Benjamini-Hochberg correction
    # _, p_values, _, _ = multipletests(p_values, method='fdr_bh')


    # Create significance matrix
    sig_matrix = np.zeros((n_models, n_models))
    k = 0
    for i in range(n_models):
        for j in range(i+1, n_models):
            sig_matrix[j, i] = -np.log10(p_values[k]) if log_scale else p_values[k]
            k += 1

    # Create mask for lower triangle
    mask = np.triu(np.ones_like(sig_matrix, dtype=bool), k=0)

    # Heatmap

    
    V = {'vmin': 0, 'vmax': 3, 'cmap': 'YlOrRd'} if log_scale else {'vmin': 0, 'vmax': 0.1, 'cmap': 'YlOrRd_r'}
    heatmap = sns.heatmap(sig_matrix, mask=mask, annot=True, fmt='.4f', 
                          xticklabels=labels_sorted, yticklabels=labels_sorted,
                          ax=ax2, cbar_ax=cax, cbar_kws={'label': '-log10(p-value)'}, **V)
    for text in ax2.texts:
        text.set_fontsize(6)


    # Set ax2 properties
    ax2.set_xticklabels(labels_sorted, rotation=45, ha='right')
    # ax2.set_yticklabels(labels_sorted, rotation=0)
    ax2.set_title('-log10(p-value)')
    ax2.set_xlabel('Model')
    ax2.set_ylabel('Model')

    # handles, labels = ax1.get_legend_handles_labels()
    # handles = handles[:-1][::-1] + [handles[-1]]
    # labels = labels[:-1][::-1] + [labels[-1]]
    # ax1.legend(handles, labels, bbox_to_anchor=(1.05, 1), loc='upper left')
    
    handles, _labels = ax1.get_legend_handles_labels()
    legend_labels = labels
    legend_handles = []
    for i, l in enumerate(labels):
        o = _labels.index(l)
        legend_handles.append(handles[o])
    if mean:
        legend_labels += ['Mean']
        legend_handles += [mean_scatter]
    ax1.legend(legend_handles, legend_labels, bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.subplots_adjust(left=0.30, right=0.70, top=0.95, bottom=0.15)

    os.makedirs('out/figs/box', exist_ok=True)
    fig_path = f'out/figs/box/{target}_{grains}_{limit}_{key}.png'
    plt.savefig(fig_path, format='png', dpi=DPI)
    print(f'saved {fig_path}')
    if show:
        print(f'{target} - {grains} - {limit} - {key}')
        plt.show()
    plt.close()

## CV 500 coarse patches/case

## EBRAINS 500 coarse patches/case

In [9]:
paired_box('cv', False, LIMIT, 'pacc')
paired_box('cv', False, LIMIT, 'recall')
# paired_box('cv', False, LIMIT, 'acc')
# paired_box('cv', False, LIMIT, 'f1')

paired_box('ebrains', False, LIMIT, 'pacc')
paired_box('ebrains', False, LIMIT, 'recall')
# paired_box('ebrains', False, LIMIT, 'acc')
# paired_box('ebrains', False, LIMIT, 'f1')

saved out/figs/box_cv_fine_500_pacc.png
saved out/figs/box_cv_fine_500_recall.png
saved out/figs/box_ebrains_fine_500_pacc.png
saved out/figs/box_ebrains_fine_500_recall.png


### EBRAINS 500 fine patches/case

In [10]:
paired_box('cv', True, LIMIT, 'pacc')
paired_box('cv', True, LIMIT, 'recall')
# paired_box('cv', True, LIMIT, 'acc')
# paired_box('cv', True, LIMIT, 'f1')

paired_box('ebrains', True, LIMIT, 'pacc')
paired_box('ebrains', True, LIMIT, 'recall')
# paired_box('ebrains', True, LIMIT, 'acc')
# paired_box('ebrains', True, LIMIT, 'f1')

saved out/figs/box_cv_coarse_500_pacc.png
saved out/figs/box_cv_coarse_500_recall.png
saved out/figs/box_ebrains_coarse_500_pacc.png
saved out/figs/box_ebrains_coarse_500_recall.png


#### fig4

This fig shows the impact of patch number per case on model performace

In [11]:
def draw_lines(target, key:str, coarse:bool, fill=False, show=False):
    labels = [
        r'Prov-GigaPath(LP)',
        r'UNI(FT)',
        r'UNI(LP)',
        r'CTransPath(FT)',
        r'CTransPath(LP)',
        r'ViT-L$\mathrm{_{IN}}$(FT)',
        r'ViT-L$\mathrm{_{IN}}$(LP)',
        r'ViT-L(RI)',
        r'ResNet-RS 50$\mathrm{_{IN}}$(FT)',
        # r'ResNet-RS 50$\mathrm{_{IN}}$(LP)',
    ]

    grains = 'coarse' if coarse else 'fine'
    key_label = {
        'pacc': 'Patch Acc.',
        'recall': 'Macro Recall',
        'acc': 'Macro Acc.',
        'f1': 'Macro F1',
    }[key]
    
    # color_map = dict(zip((label_order), plt.cm.Set2(np.linspace(0, 1, len(label_order)))))
    # color_map = dict(zip(label_order, sns.color_palette(PALETTE, n_colors=len(label_order))))
    
    limits = [10, 25, 100, 500]
    alpha = 0.05
    
    values_by_limit = []

    for limit in limits:
        values = []
        df = pd.read_excel(f'out/figs/tables/results_{grains}_{target}.xlsx', sheet_name=f'{limit}', index_col=0)


        for label in labels:
            scores = df[df['label']==label][key_label]
            mean = np.mean(scores)
            diff = stats.sem(scores)
            # CI
            # diff = stats.t.interval(1 - alpha, df=len(scores)-1, loc=mean, scale=diff)[1] - mean
            values.append([mean, mean-diff, mean+diff])
        values_by_limit.append(values)    
    values_by_cond = np.array(values_by_limit).transpose(1, 0, 2)
    
    plt.figure(figsize=(6, 7))
    x = np.arange(1, 1+len(limits))

    for i, label in enumerate(labels):
        ci_l = values_by_cond[i, :, 1]
        ci_h = values_by_cond[i, :, 2]
        # c = colors[i]
        c = color_map[label]
        plt.plot(x, values_by_cond[i, :, 0], color=c, label=label)
        if fill:
            plt.fill_between(x, ci_l, ci_h, color=c, alpha=0.1)
    
    plt.xticks(x, limits)
    plt.xlabel('Patch count per case')
    plt.ylabel(key_label)
    # plt.legend(loc='lower right')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    # plt.grid(True, linestyle='--', alpha=0.7)
    target_dataset = {
        'cv': 'Local',
        'ebrains': 'EBRAINS',
    }[target]
    plt.title(f'{target_dataset} - {key_label}')    
    plt.tight_layout()

    os.makedirs('out/figs/lines', exist_ok=True)
    fig_path = f'out/figs/lines/{target}_{grains}_{key}.png'
    plt.savefig(fig_path, format="png", dpi=DPI)
    print(f'wrote {fig_path}')
    if show:
        plt.show()
    else:
        plt.close()



In [12]:
draw_lines('cv', 'pacc', False)
draw_lines('cv', 'recall', False)
draw_lines('cv', 'acc', False)
draw_lines('cv', 'f1', False)

draw_lines('ebrains', 'pacc', False)
draw_lines('ebrains', 'recall', False)
draw_lines('ebrains', 'acc', False)
draw_lines('ebrains', 'f1', False)

wrote out/figs/lines_cv_fine_pacc.png
wrote out/figs/lines_cv_fine_recall.png
wrote out/figs/lines_cv_fine_acc.png
wrote out/figs/lines_cv_fine_f1.png
wrote out/figs/lines_ebrains_fine_pacc.png
wrote out/figs/lines_ebrains_fine_recall.png
wrote out/figs/lines_ebrains_fine_acc.png
wrote out/figs/lines_ebrains_fine_f1.png


### Fig4: EBRAINS - lines

In [13]:
draw_lines('cv', 'pacc', True)
draw_lines('cv', 'recall', True)
draw_lines('cv', 'acc', True)
draw_lines('cv', 'f1', True)

draw_lines('ebrains', 'pacc', True)
draw_lines('ebrains', 'recall', True)
draw_lines('ebrains', 'acc', True)
draw_lines('ebrains', 'f1', True)

wrote out/figs/lines_cv_coarse_pacc.png
wrote out/figs/lines_cv_coarse_recall.png
wrote out/figs/lines_cv_coarse_acc.png
wrote out/figs/lines_cv_coarse_f1.png
wrote out/figs/lines_ebrains_coarse_pacc.png
wrote out/figs/lines_ebrains_coarse_recall.png
wrote out/figs/lines_ebrains_coarse_acc.png
wrote out/figs/lines_ebrains_coarse_f1.png
